<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/llama_relik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the repository
!git clone https://github.com/richardpaulhudson/coreferee

# Change to the repository directory
%cd coreferee

# Modify the setup.cfg file
!sed -i '39s/spacy>=3.0.0,<3.6.0/spacy>=3.0.0,<3.8.0/' setup.cfg
!sed -i 's/to_version:             3.3.0/to_version:             3.8.0/g' coreferee/lang/en/config.cfg
# Verify the change
!cat setup.cfg | grep spacy

# Install the package
!pip install .

Cloning into 'coreferee'...
remote: Enumerating objects: 424, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 424 (delta 48), reused 60 (delta 37), pack-reused 325
Receiving objects: 100% (424/424), 248.60 MiB | 25.30 MiB/s, done.
Resolving deltas: 100% (173/173), done.
Updating files: 100% (153/153), done.
/content/coreferee
keywords= nlp, spacy, spacy-extension, python, machine-learning, coreference, anaphora, coreference-resolution, anaphora-resolution
  spacy>=3.0.0,<3.8.0
spacy_factories =
Processing /content/coreferee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for coreferee: filename=coreferee-1.4.1-py3-none-any.whl size=182569 sha256=71834b7e7dfefee0ae2a0d433b41c1ce954a2377f78f7c8cba496cab42718287
  Stored in directory: /tmp/pip-ephem-wheel-cache-m2s51cu_/wheels/a3/71/a1/572217811ada49a1a01439dd1902bf93f82f

In [2]:
!pip install --quiet llama-index-extractors-relik llama-index-graph-stores-neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.8/601.8 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.5/808.5 kB 38.9 MB/s eta

In [3]:
!python -m spacy download en_core_web_lg
!python -m coreferee install en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Processing ./models/en
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for coreferee-model-en: filename=coreferee_model_en-1.0.0-py3-none-any.whl size=65422507 sha256=c0d69b333a6f0e5147bd386302195956b1ca1bc7ab5fc6bb3d498cd6972b94c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-q03wo5x9/wheels/f6/be/ad/ab1b76b72291d45f7a92a9a29098c985a2fbc9ad9f97c658ac
Successfully built coreferee-model-en


In [4]:
import spacy, coreferee

coref_nlp = spacy.load('en_core_web_lg')
coref_nlp.add_pipe('coreferee')


def coref_text(text):
    coref_doc = coref_nlp(text)
    resolved_text = ""

    for token in coref_doc:
        repres = coref_doc._.coref_chains.resolve(token)
        if repres:
            resolved_text += " " + " and ".join(
                [
                    t.text
                    if t.ent_type_ == ""
                    else [e.text for e in coref_doc.ents if t in e][0]
                    for t in repres
                ]
            )
        else:
            resolved_text += " " + token.text

    return resolved_text

print(coref_text("Tomaz is so cool. He can solve various Python dependencies and not cry"))

 Tomaz is so cool . Tomaz can solve various Python dependencies and not cry


In [ ]:
import pandas as pd
from llama_index.core import Document


news = pd.read_csv("https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/news_articles.csv")
news['coref_text'] = news['text'].apply(coref_text)
documents = [Document(text=f"{row['title']}: {row['coref_text']}") for i, row in news.iterrows()]


print(coref_text(news['text'][5]))

⚠ Unexpected error in Coreferee annotating document, skipping ....
⚠ <class 'KeyboardInterrupt'>
⚠


  File "/usr/local/lib/python3.10/dist-packages/coreferee/manager.py", line 144, in __call__
    self.annotator.annotate(doc)
  File "/usr/local/lib/python3.10/dist-packages/coreferee/annotation.py", line 378, in annotate
    self.tendencies_analyzer.score(doc, self.thinc_ensemble)
  File "/usr/local/lib/python3.10/dist-packages/coreferee/tendencies.py", line 348, in score
    document_pair_info = DocumentPairInfo.from_doc(doc, self, ENSEMBLE_SIZE)
  File "/usr/local/lib/python3.10/dist-packages/coreferee/tendencies.py", line 546, in from_doc
    static_info.extend(tendencies_analyzer.get_position_map(mention, doc))
  File "/usr/local/lib/python3.10/dist-packages/coreferee/tendencies.py", line 219, in get_position_map
    [
  File "/usr/local/lib/python3.10/dist-packages/coreferee/tendencies.py", line 223, in <listcomp>
    and len(list(token_in_sentence.ancestors))


⚠ Unexpected error in Coreferee annotating document, skipping ....
⚠ <class 'KeyboardInterrupt'>
⚠


  File "/usr/local/lib/python3.10/dist-packages/coreferee/manager.py", line 144, in __call__
    self.annotator.annotate(doc)
  File "/usr/local/lib/python3.10/dist-packages/coreferee/annotation.py", line 377, in annotate
    self.rules_analyzer.initialize(doc)
  File "/usr/local/lib/python3.10/dist-packages/coreferee/rules.py", line 252, in initialize
    token._.coref_chains.temp_quote_array = working_quote_array[:]
  File "spacy/tokens/token.pyx", line 163, in spacy.tokens.token.Token._.__get__
  File "/usr/local/lib/python3.10/dist-packages/spacy/tokens/underscore.py", line 22, in __init__
    def __init__(
